# TD Final : Analyse des Avis et Alertes ANSSI avec Enrichissement des CVE
*Solution proposée par Laura Damas, Camille Dommergue, Maxime Cerruti et Elodie Duflaut*

Importation des libraries

In [1]:
import pandas as pd
import feedparser as fp
import requests as req
import re

Etape 1 - Extraction des Flux RSS

In [2]:
url_avis = "https://cert.ssi.gouv.fr/avis/feed"
url_alerte = "https://cert.ssi.gouv.fr/alerte/feed"

rss_feed_avis = fp.parse(url_avis)
rss_feed_alerte = fp.parse(url_alerte)

# Check if the feed was parsed correctly
if rss_feed_avis.bozo:
    raise ValueError("Failed to parse RSS AVIS feed. Please check the URL.\n")
elif rss_feed_alerte.bozo:
    raise ValueError("Failed to parse RSS ALERTE feed. Please check the URL.\n")
else:
    print(f"Successfully parsed RSS AVIS feed from {url_avis}\n")
    print(f"Successfully parsed RSS ALERTE feed from {url_alerte}\n")

rows = []

def get_bulletin_id(ent_id):
    parts = ent_id.split('/')
    bulletin_string = parts[-2]
    return bulletin_string

def clean_title(title_string):
    """
    Removes any text enclosed within square brackets [] or parentheses ()
    from a given string.
    """

    # Remove text in parentheses
    cleaned_string = re.sub(r"\(.*?\)", "", title_string)

    # Remove text in square brackets from the already cleaned string
    cleaned_string = re.sub(r"\[.*?\]", "", cleaned_string)

    # Remove any extra spaces that might result from the removal
    cleaned_string = re.sub(r"\s\s+", " ", cleaned_string).strip()

    return cleaned_string



# Loop through RSS feeds entries and collect data
for entry in rss_feed_avis.entries:
    
    rows.append({
        "Id": get_bulletin_id(entry.id),
        "Title": clean_title(entry.title),
        "Type": "Avis",
        "Link": entry.link,
        "Summary": entry.summary,
        "Published": entry.published
    })

for entry in rss_feed_alerte.entries:
    
    rows.append({
        "Id": get_bulletin_id(entry.id),
        "Title": clean_title(entry.title),
        "Type": "Alerte",
        "Link": entry.link,
        "Summary": entry.summary,
        "Published": entry.published
    })

# Convert list of rows into a DataFrame
df_flux_rss = pd.DataFrame(rows, columns=["Id", "Title", "Type", "Link", "Published", "Summary"])

# Convert 'published' to datetime, handling errors
df_flux_rss["Published"] = pd.to_datetime(df_flux_rss["Published"], format="%a, %d %b %Y %H:%M:%S %z", errors='coerce')

# Order the DataFrame by the "published" column
df_flux_rss = df_flux_rss.sort_values(by="Published", ascending=False)

Successfully parsed RSS AVIS feed from https://cert.ssi.gouv.fr/avis/feed

Successfully parsed RSS ALERTE feed from https://cert.ssi.gouv.fr/alerte/feed



In [3]:
print("\nInfos", df_flux_rss.info())
print("\nHead", df_flux_rss.head())
print("\nTail", df_flux_rss.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 39 to 40
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   Id         80 non-null     object             
 1   Title      80 non-null     object             
 2   Type       80 non-null     object             
 3   Link       80 non-null     object             
 4   Published  80 non-null     datetime64[ns, UTC]
 5   Summary    80 non-null     object             
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 4.4+ KB

Infos None

Head                       Id                                              Title  \
39  CERTFR-2025-AVI-0515                         Vulnérabilité dans Grafana   
38  CERTFR-2025-AVI-0514                 Vulnérabilité dans PostgreSQL JDBC   
37  CERTFR-2025-AVI-0513       Multiples vulnérabilités dans Microsoft Edge   
35  CERTFR-2025-AVI-0511  Multiples vulnérabilités dans les produits Google   
31  CERTF

Etape 2 - Extraction des CVE

In [4]:
def extract_cve_from_link(link):
    json_link = link.rstrip("/") + "/json/" 
    response = req.get(json_link)
    if response.status_code != 200:
        print(f"Erreur: impossible d'accéder à {json_link}")
        return []

    try:
        data = response.json()
    except ValueError:
        print(f"Le contenu de {json_link} n'est pas un JSON valide.")
        return []
    
    cve_list  = [item['name'] for item in data["cves"]]
    
    return cve_list

In [5]:
#extract CVEs from each link in the DataFrame & store them in a new column
df_flux_rss["CVE"] = df_flux_rss["Link"].apply(extract_cve_from_link)

In [6]:
def display_cve_info(bulletin_id, df):
    """
    Displays the CVE information for a given bulletin ID from the DataFrame.
    """
    # Ensure the 'id' column is cleaned of leading/trailing whitespace before comparison
    filtered_data = df[df["Id"].str.strip() == bulletin_id.strip()]

    print(f"\n--- CVE Info for ID: {bulletin_id} ---")

    if not filtered_data.empty:
        # Get the 'cve' column data for the filtered rows
        cve_info_list = filtered_data["CVE"].iloc[0] # Assuming 'id' is unique, take the first match

        if cve_info_list: # Check if the list of CVEs is not empty
            print("CVEs found:")
            for cve_entry in cve_info_list:
                if isinstance(cve_entry, dict) and 'name' in cve_entry:
                    print(f"- Name: {cve_entry['name']}")
                    if 'url' in cve_entry:
                        print(f"  URL: {cve_entry['url']}")
                else:
                    print(f"  Malformed CVE entry: {cve_entry}")
        else:
            print(f"No CVEs listed for bulletin ID: {bulletin_id}.")
    else:
        print(f"No entry found for bulletin ID: {bulletin_id}.")

print("\nTest CVE | CERTFR-2025-ALE-008:\n", display_cve_info("CERTFR-2025-ALE-008", df_flux_rss))
print("\nTest CVE | CERTFR-2025-AVI-0512:\n",display_cve_info("CERTFR-2025-AVI-0512", df_flux_rss))


--- CVE Info for ID: CERTFR-2025-ALE-008 ---
CVEs found:
  Malformed CVE entry: CVE-2025-49113

Test CVE | CERTFR-2025-ALE-008:
 None

--- CVE Info for ID: CERTFR-2025-AVI-0512 ---
CVEs found:
  Malformed CVE entry: CVE-2025-0917
  Malformed CVE entry: CVE-2018-19361
  Malformed CVE entry: CVE-2023-29483
  Malformed CVE entry: CVE-2021-33036
  Malformed CVE entry: CVE-2019-17267
  Malformed CVE entry: CVE-2024-22201
  Malformed CVE entry: CVE-2025-27516
  Malformed CVE entry: CVE-2018-14719
  Malformed CVE entry: CVE-2020-9546
  Malformed CVE entry: CVE-2024-28757
  Malformed CVE entry: CVE-2025-47944
  Malformed CVE entry: CVE-2024-12797
  Malformed CVE entry: CVE-2025-30065
  Malformed CVE entry: CVE-2025-27219
  Malformed CVE entry: CVE-2024-25638
  Malformed CVE entry: CVE-2023-45853
  Malformed CVE entry: CVE-2017-9047
  Malformed CVE entry: CVE-2020-9548
  Malformed CVE entry: CVE-2023-45288
  Malformed CVE entry: CVE-2023-45178
  Malformed CVE entry: CVE-2024-47076
  Malformed 

Etape 3 - Enrichissement des CVE

In [7]:
def connexion_to_apis(cve_id, verbose=None):
    # CVE API
    url = f"https://cveawg.mitre.org/api/cve/{cve_id}"
    response = req.get(url)

    if response.status_code != 200:
        if verbose:
            print(f"Erreur: impossible d'accéder à l'API CVE pour {cve_id}. Statut: {response.status_code}")
        return None
    try:
        data = response.json()
    except req.exceptions.JSONDecodeError:
        if verbose:
            print(f"Erreur: Impossible de décoder la réponse JSON de l'API CVE pour {cve_id}.")
        return None

    # Description
    description = None
    desc_list = data.get("containers", {}).get("cna", {}).get("descriptions", [])
    for d in desc_list:
        if d.get("lang") == "en":
            description = d.get("value")
            break

    # CVSS score & base severity
    cvss_score = None
    base_severity = None
    cvss_score_float = None
    metrics = data.get("containers", {}).get("cna", {}).get("metrics", [])
    if metrics:
        for metric in metrics:
            for key in ["cvssV3_1", "cvssV3_0", "cvssV2"]:
                if key in metric:
                    cvss_score = metric[key].get("baseScore")
                    base_severity = metric[key].get("baseSeverity")
            if cvss_score is not None:
                break

    # CWE(s)
    cwes = []
    problem_types = data.get("containers", {}).get("cna", {}).get("problemTypes", [])
    for pt in problem_types:
        for desc in pt.get("descriptions", []):
            if desc.get("lang") == "en":
                cwes.append(desc.get("cweId"))

    # Produits & versions affectés
    affected_products = []
    affected = data.get("containers", {}).get("cna", {}).get("affected", [])
    for prod in affected:
        vendor = prod.get("vendor")
        product = prod.get("product")
        versions = prod.get("versions")
        version_lst = []
        for elt in versions:
            version = elt.get("version", None)
            if version == "unspecified":
                version = f"Less than {elt['lessThan']}"
            version_lst.append(version)
        affected_prod = {
            "vendor": vendor,
            "product": product,
            "versions": version_lst,
        }
        affected_products.append(affected_prod)


    # EPSS API
    url = f"https://api.first.org/data/v1/epss?cve={cve_id}"
    response = req.get(url)

    if response.status_code != 200:
        if verbose:
            print(f"Erreur: impossible d'accéder à l'API EPSS pour {cve_id}. Statut: {response.status_code}")
        return None
    try:
        data = response.json()
    except req.exceptions.JSONDecodeError:
        if verbose:
            print(f"Erreur: Impossible de décoder la réponse JSON de l'API EPSS pour {cve_id}.")
        return None

    # EPSS score
    infos = data["data"]
    epss_score = None
    epss_percentile = None
    if infos:
        epss_info = infos[0]
        epss_score = epss_info.get("epss", None)
        epss_percentile = epss_info.get("percentile", None)

    return {
        "CVE_id": cve_id,
        "Description": description,
        "CVSS_score": cvss_score,
        "Base_severity": base_severity,
        "CWE": cwes,
        "Affected_products": affected_products,
        "EPSS_score": epss_score,
        "EPSS_percentile": epss_percentile
    }


In [9]:
set_cve_uniques = set() #unique values

cve_list=df_flux_rss["CVE"]
for cve_list in df_flux_rss["CVE"]:
    for cve_id in cve_list: 
        set_cve_uniques.add(cve_id)

In [10]:
liste_cve_info = []

for cve_id in set_cve_uniques:
    data = connexion_to_apis(cve_id)  
    #print(f"\n{data} for {cve_id}")
    if data:
        liste_cve_info.append(data)

TypeError: 'NoneType' object is not iterable

In [ ]:
df_cves = pd.DataFrame(liste_cve_info)

print(df_cves.info())
print(df_cves.head())
print(df_cves.tail())

Etape 4 - Consolidation des Données

In [ ]:
''' Current dataframes and their contents:
df_flux_rss : Id, Title, Type, Link, Published, Summary, CVE(list)
df_cves : CVE_id, Description, CVSS_score, Base_severity, CWE, Affected_products(dictionnaire), EPSS_score, EPSS_percentile

Wanted dataframe:
df_consolidated : Id, Link, Title, Type, Published, Summary, CVE_id, Description, CVSS_score, Base_severity, CWE, Affected_products, EPSS_score, EPSS_percentile
'''

df_flux_rss_exploded = df_flux_rss.explode('CVE')

df_consolidated = pd.merge(df_flux_rss_exploded, df_cves, left_on="CVE", right_on="CVE_id", how="left")
# Reorder the columns in the desired order 
df_consolidated = df_consolidated[[
    "Id", "Link", "Title", "Type", "Published", "Summary",
    "CVE_id", "Description", "CVSS_score", "Base_severity",
    "CWE", "Affected_products", "EPSS_score", "EPSS_percentile"
]]

print("\nConsolidated DataFrame Info:")
print(df_consolidated.info())
print("\nConsolidated DataFrame Head:")
print(df_consolidated.head())
print("\nConsolidated DataFrame Tail:") 
print(df_consolidated.tail())